In [92]:
import cv2
import numpy as np
import imutils
import random
import pytesseract

In [93]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [94]:
def denoise_filter2D(image, kernel=None):
    if kernel is None:
        kernel = np.ones((2,2), np.float32)/4    
        
    denoised_image = cv2.filter2D(image, -1, kernel)
    
    return denoised_image

In [95]:
def standard_thresholding(image):
    
    ret3,threshold_image = cv2.threshold(image,170,255,cv2.THRESH_BINARY)
    
    return threshold_image

In [96]:
def get_receipt_contour(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    h_perimeter = 0
    h_contour = []
    for cnt in contours:
        cnt_perimeter = cv2.arcLength(cnt,True)
        if cnt_perimeter > h_perimeter:
            h_perimeter = cnt_perimeter
            h_contour = cnt
            
    return h_contour

In [97]:
def adjust_receipt_rotation(receipt_contour, img):   
    rect = cv2.minAreaRect(receipt_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    angle = rect[-1]

    if angle < -45:
        angle =  90 + angle

    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h),flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


In [98]:
def prepare_image(gray_scaled_image):
    no_noise_image = denoise_filter2D(gray_scaled_image)

    threshold_image = standard_thresholding(no_noise_image)

    no_noise_image = denoise_filter2D(threshold_image)
    return no_noise_image

In [99]:
IMAGEPATH = "/home/simone/Desktop/receipt_test.jpg"

image = cv2.imread(IMAGEPATH)
gray_scaled_image = cv2.imread(IMAGEPATH, cv2.IMREAD_GRAYSCALE)

resize_param = 500

image = image_resize(image,resize_param)
gray_scaled_image = image_resize(gray_scaled_image,resize_param)



In [100]:
useful_image = prepare_image(gray_scaled_image)

In [101]:
edges = cv2.Canny(useful_image,100,200)


In [102]:
receipt_contour = get_receipt_contour(edges)
        
rotated = adjust_receipt_rotation(receipt_contour, useful_image)
   
#_ = cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

In [103]:

receipt_contour = get_receipt_contour(rotated)
x,y,w,h = cv2.boundingRect(receipt_contour)
cropped = rotated[y:y+h, x:x+w]

In [104]:
pytesseract.image_to_string(cropped)

'Main Street Revtaurent\n11} Gumiuers Orive\nSuite 528\npalo Alto California 9430t\n‘S25 “1628095,\n\nFe, 04/07/2017 11:36 Ad\n\nMerchant 10: 9nqyxvufde\nTerminal 10: a\n\nTransaction 10: e6ds9set\nType: CREDIT\n\nPURCHASE:\nNumber 29100009000000004 1\nEntry Mode ‘Smiped\nCard Type: DISCOVER\n\nResponse: APPROVED\n‘Approval Code: 819543\n\nSub Total sos 25.23\nTip: 3.78\n\nYotal uso 29.01\n\nThanks for suppor cing\nLocal business |\n\nTHANK YOU'

In [105]:

#cv2.imshow("gray image", gray_image_scaled)
cv2.imshow("starting image", image)
#cv2.imshow("threshold image", threshold_image)
cv2.imshow("no noise image", useful_image)
#cv2.imshow("edges", edges)

cv2.imshow("rotated image", rotated)
cv2.imshow("cropped image", cropped)







cv2.waitKey()
cv2.destroyAllWindows()
